In [1]:
!pip install pymongo[srv,tls]
!pip install dnspython==2.0.0

     |████████████████████████████████| 208 kB 492 kB/s eta 0:00:01
  Attempting uninstall: dnspython
    Found existing installation: dnspython 1.16.0
    Uninstalling dnspython-1.16.0:
      Successfully uninstalled dnspython-1.16.0


In [2]:
import pymongo
import pandas as pd

In [3]:
user = 'Estudante'
pwd = 'Estudante20202'
connection_string = 'mongodb+srv://{}:{}@bdrnr.mw4lf.mongodb.net/bdrnr_db?retryWrites=true&w=majority'.format(user, pwd)
client = pymongo.MongoClient(connection_string)
db = client.bdrnr_db

In [4]:
# Consulta 1

# Para nascidos neste século (DATA_NASCIMENTO > "2001"), listar AGENTE_CAUSADOR_ACIDENTE, DATA_NASCIMENTO e SEXO,
# ordenados por DATA_NASCIMENTO descendente, SEXO e AGENTE_CAUSADOR_ACIDENTE.

resultado = db.G01.find({'DATA_NASCIMENTO': {'$gt': '2001'}},
                      {'AGENTE_CAUSADOR_ACIDENTE':1,
                       'DATA_NASCIMENTO':1,
                       'SEXO':1,
                       '_id':0}).sort([('DATA_NASCIMENTO', -1),
                                        ('SEXO', 1),
                                        ('AGENTE_CAUSADOR_ACIDENTE', 1)])
df = pd.DataFrame(list(resultado))
df

,AGENTE_CAUSADOR_ACIDENTE,SEXO,DATA_NASCIMENTO
0,"Veiculo, Nic",Masculino,2004-07-14
1,Queda de Pes. em Mes,Feminino,2003-01-27
2,Empilhadeira,Masculino,2002-04-13
3,"Caixa, Engradado, Ca",Masculino,2002-01-17
4,"Tesoura, Tesourao- F",Masculino,2002-01-03
5,Superficie e Estrutu,Masculino,2001-09-11
6,Metal - Inclui Liga,Masculino,2001-09-09
7,Escada Permanente Cu,Feminino,2001-07-28
8,Escada Permanente Cu,Feminino,2001-06-29
9,"Faca, Facao- Ferrame",Masculino,2001-05-27


In [5]:
# Consulta 2

# Ordenados por número de acidentes em ordem decrescente, listar MUNICIPIO, POPULACAO e número de acidentes, dos 20 municípios com mais acidentes.

resultado = db.G01.aggregate([{"$group":{"_id": {"MUNICIPIO":"$MUNICIPIO",
                                                 "POPULACAO":"$POPULACAO"},
                                         "NUMERO_ACIDENTES": {"$sum":1}}},
                              {"$sort":{"NUMERO_ACIDENTES" : -1}},
                              {"$limit":20}])

print("{:22}\t{:12}\t{:18}".format("Município", "População", "Número acidentes"))
print("{:_<22}\t{:_<12}\t{:_<18}".format("_", "_", "_"))
for documento in resultado:    
    print("{:22}\t{:12}\t{:18}".format(documento["_id"]["MUNICIPIO"],
                                       documento["_id"]["POPULACAO"],
                                       documento["NUMERO_ACIDENTES"]))

Município             	População   	Número acidentes  
______________________	____________	__________________
Porto Alegre          	     1479101	               297
Caxias do Sul         	      504069	               107
Passo Fundo           	      201767	                63
Canoas                	      344957	                43
Santa Maria           	      280505	                40
Erechim               	      105059	                35
Lajeado               	       82951	                34
Sao Leopoldo          	      234947	                31
Gravatai              	      279398	                29
Santa Cruz do Sul     	      129427	                26
Marau                 	       43435	                22
Novo Hamburgo         	      246452	                20
Rio Grande            	      210005	                19
Cachoeirinha          	      129307	                19
Bento Gonçalves       	      119049	                17
Santa Rosa            	       72919	                16
Carazinho 

In [6]:
# Consulta 3

# Ordenados em ordem decrescente por número de acidentes,
# listar as 15 primeiras denominações das atividades econômicas (CNAE_DENOMINACAO), municípios e número de acidentes.
# (dica, agrupar por CNAE_DENOMINACAO e MUNICIPIO)

resultado = db.G01.aggregate([{"$group":{"_id": {"CNAE_DENOMINACAO":"$CNAE_DENOMINACAO",
                                                 "MUNICIPIO":"$MUNICIPIO"},
                                         "NUMERO_ACIDENTES": {"$sum":1}}},
                              {"$sort":{"NUMERO_ACIDENTES" : -1}},
                              {"$limit":15}])

print("{:116}\t{:22}\t{:18}".format("CNAE Denominação", "Município", "Número acidentes"))
print("{:_<116}\t{:_<22}\t{:_<18}".format("_", "_", "_"))
for documento in resultado:    
    print("{:116}\t{:22}\t{:18}".format(documento["_id"]["CNAE_DENOMINACAO"],
                                       documento["_id"]["MUNICIPIO"],
                                       documento["NUMERO_ACIDENTES"]))

CNAE Denominação                                                                                                    	Município             	Número acidentes  
____________________________________________________________________________________________________________________	______________________	__________________
Atividades de atendimento hospitalar                                                                                	Porto Alegre          	               132
Atividades de atendimento hospitalar                                                                                	Passo Fundo           	                22
Abate de suinos, aves e outros pequenos animais                                                                     	Lajeado               	                18
Atividades de atendimento hospitalar                                                                                	Caxias do Sul         	                16
Atividades de atencao a saude humana nao espec

In [7]:
# Consulta 4

# Ordenados por mês/ano, denominações das atividades econômicas e número de acidentes,
# filtrando por agente causador de acidente, sexo e atividade econômica.
# (escolham uma trinca (agente causador de acidente, sexo e atividade econômica) que permita a consulta retornar entre 5 e 30 linhas).

resultado = db.G01.aggregate([{"$match":{"AGENTE_CAUSADOR_ACIDENTE":"Agente Infeccioso ou",
                                         "SEXO":"Feminino",
                                         "CNAE_DENOMINACAO":"Atividades de atendimento hospitalar"}},
                              {"$group":{"_id": {"MES_ANO_ACIDENTE":"$MES_ANO_ACIDENTE",
                                                 "CNAE_DENOMINACAO":"$CNAE_DENOMINACAO"},
                                         "NUMERO_ACIDENTES": {"$sum":1}}},
                              {"$sort":{"_id.MES_ANO_ACIDENTE":1}}])

print("{:20}\t{:36}\t{:18}".format("Mês/Ano Acidente", "CNAE Denominação", "Número Acidentes"))
print("{:_<20}\t{:_<36}\t{:_<18}".format("_", "_", "_"))
for documento in resultado:    
    print("{:20}\t{:36}\t{:18}".format(documento["_id"]["MES_ANO_ACIDENTE"],
                                       documento["_id"]["CNAE_DENOMINACAO"],
                                       documento["NUMERO_ACIDENTES"]))

Mês/Ano Acidente    	CNAE Denominação                    	Número Acidentes  
____________________	____________________________________	__________________
2019-10-01          	Atividades de atendimento hospitalar	                 7
2019-11-01          	Atividades de atendimento hospitalar	                10
2019-12-01          	Atividades de atendimento hospitalar	                 5
2020-01-01          	Atividades de atendimento hospitalar	                 6
2020-02-01          	Atividades de atendimento hospitalar	                 2
2020-03-01          	Atividades de atendimento hospitalar	                 5


In [8]:
# Consulta 5

# Listar os 10 municípios e respectivas populações com as maiores taxas de acidentes por 100.000 habitantes.

resultado = db.G01.aggregate([{"$group":{"_id": {"MUNICIPIO":"$MUNICIPIO",
                                                 "POPULACAO":"$POPULACAO"},
                                         "NUMERO_ACIDENTES": {"$sum":1}}},
                              {"$project": {"TAXA_ACIDENTES_100MIL_HAB": {"$multiply":[{"$divide": ["$NUMERO_ACIDENTES", "$_id.POPULACAO"]}, 100000]}}},
                              {"$sort":{"TAXA_ACIDENTES_100MIL_HAB" : -1}},
                              {"$limit":10}])

print("{:22}\t{:12}\t{:28}".format("Município", "População", "Taxa acidentes 100 mil hab."))
print("{:_<22}\t{:_<12}\t{:_<28}".format("_", "_", "_"))
for documento in resultado:    
    print("{:22}\t{:12}\t{:28}".format(documento["_id"]["MUNICIPIO"],
                                       documento["_id"]["POPULACAO"],
                                       documento["TAXA_ACIDENTES_100MIL_HAB"]))

Município             	População   	Taxa acidentes 100 mil hab. 
______________________	____________	____________________________
Vila Langaro          	        2102	          142.72121788772597
Paulo Bento           	        2287	            87.4508089199825
Nova Araça            	        4690	           85.28784648187633
Nova Bassano          	        9824	           71.25407166123779
Carlos Gomes          	        1404	           71.22507122507123
Vila Maria            	        4353	           68.91798759476224
Westfalia             	        2998	           66.71114076050699
Tio Hugo              	        3004	           66.57789613848203
Coqueiro Baixo        	        1507	              66.35700066357
Nao-Me-Toque          	       17484	           57.19514985129261
